<a href="https://colab.research.google.com/github/jppasso/Computer-Vision-University-Assignment-CAPG002/blob/main/Vegetables3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

# Import Kaggle Dataset

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"joaopassos","key":"503dc7e31a72caa4a7b8fe1f3b41074f"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              233KB  2021-08-16 08:33:52          10737  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           6558  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           2475  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-07-03 18:37:20           2654  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Jo

In [ ]:
! kaggle datasets download -d raghavrpotdar/fresh-and-stale-images-of-fruits-and-vegetables

100% 1.40G/1.41G [00:13<00:00, 91.2MB/s]
100% 1.41G/1.41G [00:13<00:00, 108MB/s] 


In [ ]:
! mkdir vegetables

In [ ]:
! unzip fresh-and-stale-images-of-fruits-and-vegetables.zip -d vegetables

Streaming output truncated to the last 5000 lines.
  inflating: vegetables/stale_banana/rotated_by_15_Screen Shot 2018-06-12 at 9.02.09 PM.png  
  inflating: vegetables/stale_banana/rotated_by_15_Screen Shot 2018-06-12 at 9.02.15 PM.png  
  inflating: vegetables/stale_banana/rotated_by_15_Screen Shot 2018-06-12 at 9.02.24 PM.png  
  inflating: vegetables/stale_banana/rotated_by_15_Screen Shot 2018-06-12 at 9.02.32 PM.png  
  inflating: vegetables/stale_banana/rotated_by_15_Screen Shot 2018-06-12 at 9.02.38 PM.png  
  inflating: vegetables/stale_banana/rotated_by_15_Screen Shot 2018-06-12 at 9.06.21 PM.png  
  inflating: vegetables/stale_banana/rotated_by_15_Screen Shot 2018-06-12 at 9.06.27 PM.png  
  inflating: vegetables/stale_banana/rotated_by_15_Screen Shot 2018-06-12 at 9.06.33 PM.png  
  inflating: vegetables/stale_banana/rotated_by_15_Screen Shot 2018-06-12 at 9.06.40 PM.png  
  inflating: vegetables/stale_banana/rotated_by_15_Screen Shot 2018-06-12 at 9.06.56 PM.png  
  inflati

# Libraries Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import pathlib
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

print(tf.__version__)

2.5.0


# Data Set Up

Set Data Directory

In [ ]:
data_dir = pathlib.Path('/content/vegetables')
image_count = len(list(data_dir.glob('*/*.png')))
print(image_count)

10144


Data Exploration

In [ ]:
fresh_apples = list(data_dir.glob('fresh_apple/*'))
PIL.Image.open(str(fresh_apples[0]))

In [ ]:
PIL.Image.open(str(fresh_apples[1]))

In [ ]:
fresh_bananas = list(data_dir.glob('fresh_banana/*'))
PIL.Image.open(str(fresh_bananas[0]))

In [ ]:
PIL.Image.open(str(fresh_bananas[1]))

Load Parameters

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

val_split = 0.2
seed_ref = 123

Dataset Split

In [ ]:
# Standard Keras Tutorial Pre-processing
# Training Dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=val_split,
  subset="training",
  seed=seed_ref,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Validation Dataset
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=val_split,
  subset="validation",
  seed=seed_ref,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 14682 files belonging to 12 classes.
Using 11746 files for training.
Found 14682 files belonging to 12 classes.
Using 2936 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['fresh_apple', 'fresh_banana', 'fresh_bitter_gourd', 'fresh_capsicum', 'fresh_orange', 'fresh_tomato', 'stale_apple', 'stale_banana', 'stale_bitter_gourd', 'stale_capsicum', 'stale_orange', 'stale_tomato']


Visualize Data

In [ ]:
# First 9 Images in the Training Dataset
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

Standardize Data


In [ ]:
# Preprocess MobileNetV2
train_ds = train_ds.map(lambda x, y: (tf.keras.applications.mobilenet_v2.preprocess_input(x), y))
val_ds = val_ds.map(lambda x, y: (tf.keras.applications.mobilenet_v2.preprocess_input(x), y))

In [ ]:
# Preprocess EfficientNetB0
# None Required

In [ ]:
# Preprocess DenseNet121
#Crashed
train_ds = train_ds.map(lambda x, y: (tf.keras.applications.densenet.preprocess_input(x), y))
val_ds = val_ds.map(lambda x, y: (tf.keras.applications.densenet.preprocess_input(x), y))

In [ ]:
# Preprocess NasNetMobile
train_ds = train_ds.map(lambda x, y: (tf.keras.applications.nasnet.preprocess_input(x), y))
val_ds = val_ds.map(lambda x, y: (tf.keras.applications.nasnet.preprocess_input(x), y))

Configure For Performance

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Trained Model Application

# Model

Create Model

In [ ]:
num_classes = 12

In [ ]:
# Standard Keras Tutorial Model
model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
# MobileNet V2 Model
model = tf.keras.applications.MobileNetV2(
    input_shape=None,
    alpha=1.0,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=num_classes,
    classifier_activation="softmax",
)

In [ ]:
# EfficientNet B0 Model
model = tf.keras.applications.EfficientNetB0(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=num_classes,
    classifier_activation="softmax",
)

In [ ]:
# DenseNet121
model = tf.keras.applications.DenseNet121(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=num_classes,
)

In [ ]:
# NasNetMobile
model = tf.keras.applications.NASNetMobile(
    input_shape=None,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=num_classes,
)

Compile Model

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Model Summary

In [ ]:
model.summary()

Model: "NASNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 111, 111, 32) 0           stem_bn1[0][0]                   
_____________________________________________________________________________________________

Model Training

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4930: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


368/368 [==============================] - 3832s 10s/step - loss: 0.6867 - accuracy: 0.7686 - val_loss: 15.4438 - val_accuracy: 0.0409
Epoch 2/10
368/368 [==============================] - 3725s 10s/step - loss: 0.3500 - accuracy: 0.8805 - val_loss: 28.8613 - val_accuracy: 0.0661
Epoch 3/10
368/368 [==============================] - 3812s 10s/step - loss: 0.2335 - accuracy: 0.9191 - val_loss: 12.1926 - val_accuracy: 0.2084
Epoch 4/10
368/368 [==============================] - 3851s 10s/step - loss: 0.1961 - accuracy: 0.9334 - val_loss: 7.4110 - val_accuracy: 0.3202
Epoch 5/10
368/368 [==============================] - 3835s 10s/step - loss: 0.1491 - accuracy: 0.9486 - val_loss: 9.9695 - val_accuracy: 0.2650
Epoch 6/10
368/368 [==============================] - 3821s 10s/step - loss: 0.1254 - accuracy: 0.9550 - val_loss: 13.0046 - val_accuracy: 0.3021
Epoch 7/10
106/368 [=======>......................] - ETA: 44:52 - loss: 0.1197 - accuracy: 0.9602

Training Results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

NameError: ignored

Predict on new data

In [ ]:
stale_banana_path = '/content/vegetables/stale_banana/rotated_by_15_Screen Shot 2018-06-12 at 8.47.28 PM.png'

img = keras.preprocessing.image.load_img(
    stale_banana_path, target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)